# Zero-shot NER

**Цель**: извлечь именованные сущности из заданного текста, используя предобученные BERT-like модели с Hugging Face.

In [1]:
import re
import pymorphy3 as pm3

import pandas as pd

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

from ipymarkup import show_span_box_markup
from tqdm.notebook import tqdm

import wikipedia

### Data Extraction

Сохраним документ, пользуясь API Wikipedia:

In [2]:
wikipedia.set_lang('ru')

In [3]:
title = 'Capcom'

article = wikipedia.page(title).content
with open('wiki_src.txt', 'w', encoding='utf-8') as f:
    f.write(article)

print(article)

Capcom Co., Ltd. (яп. 株式会社カプコン Кабусики-гайся Капукон) — японская корпорация, один из крупнейших в мире разработчиков и издателей компьютерных видеоигр, со штаб-квартирой в городе Осака. Корпорация была основана в 1979 году под названием Japan Capsule Computers, как компания по производству и дистрибуции электронных игровых машин. Настоящее название это аббревиатура от Capsule и Computers. На приставки 3-4 поколения (NES, Sega, SNES) компания создавала игры по мотивам мультфильмов Walt Disney.
На сегодняшний день компания известна как владелец нескольких крупных игровых франшиз, среди которых: Resident Evil, Devil May Cry, Street Fighter, Mega Man, Lost Planet, Monster Hunter и Ace Attorney.
Capcom также активно работает как издатель и локализатор западных продуктов на востоке, примером может быть «X2» от Team 17 и Grand Theft Auto.


== Талисман ==
Оригинальным талисманом Capcom является Капитан Коммандо, супергерой, одетый в футуристическую броню. Его имя образовано от названия компа

### Preprocessing

In [4]:
len(article)

7947

Так как текст достаточно длинный, поделим его на чанки:

In [5]:
chunks = [] # [(text_chunk, chunk_len)]
chunk_start = 0

title_pattern = re.compile(r'==.*==')
for match in title_pattern.finditer(article):
    chunk_end = match.start()

    # saving only non-empty chunks
    if chunk_end - chunk_start > 0:
        chunks.append((chunk_start, chunk_end))

    chunk_start = match.end()

# saving last chunk
if len(article) - chunk_start > 0:
    chunks.append((chunk_start, len(article)))

chunks

[(0, 847),
 (861, 1912),
 (1931, 2845),
 (2861, 3335),
 (3371, 5044),
 (5070, 6811),
 (6828, 6831),
 (6863, 7228),
 (7244, 7247),
 (7262, 7617),
 (7629, 7947)]

Будем возвращать сущности в нормальной форме:

In [6]:
lemmatizer = pm3.MorphAnalyzer(lang='ru')

In [7]:
# hard-to-lemmatize exceptions
entity_exceptions = {
    'Минами',
}

In [8]:
def lemmatize(entity: str) -> str:
    """Lemmatizes named entity."""
    # do not change non-russian entities
    match = re.search(r'[а-яА-Я]', entity)
    if match is None or match.start() > 0:
        return entity

    # split the entity into tokens
    tokens = entity.split()

    # check if the last token is an exception
    revert_flag = False
    if tokens[-1] in entity_exceptions:
        token_backup = tokens[-1]
        revert_flag = True

    # lemmatize the last token
    tokens[-1] = lemmatizer.parse(tokens[-1])[0].normal_form
    gender = lemmatizer.parse(tokens[-1])[0].normalized.tag.gender
    number = lemmatizer.parse(tokens[-1])[0].normalized.tag.number

    # inflect the first token to suit the last one
    inflection = {'nomn', number}
    if number != 'plur':
        if gender is None:
            return entity
        inflection.add(gender)
    tokens[0] = lemmatizer.parse(tokens[0])[0].inflect(inflection).word

    # revert last token changes if it is an exception
    if revert_flag:
        tokens[-1] = token_backup

    # merge tokens
    return ' '.join(tokens).title()

In [9]:
lemmatize('Объединенном Королевстве')

'Объединённое Королевство'

### Model Testing 

Хоть текст и на русском языке, в нем встречаются сущности на русском и английском языках (и последних явно больше). Поэтому имеет смысл рассмотреть и мультиязычные модели, обученные для en/ru.

Модели:
1. Babelscape/wikineural-multilingual-ner -> mbert (177 миллионов параметров, 9 языков), fine-tuned на WikiNEuRal;
2. xlm-roberta-large-finetuned-conll03-english -> distilbert-base-multilingual-cased (561 миллион параметров, 100 языков), fine-tuned на conll2003;
3. denis-gordeev/rured2-ner-microsoft-mdeberta-v3-base -> mdeberta (278 миллионов параметров, русский язык), fine-tuned на RURED2.

In [10]:
checkpoints = [
    'Babelscape/wikineural-multilingual-ner',
    'xlm-roberta-large-finetuned-conll03-english',
    'denis-gordeev/rured2-ner-microsoft-mdeberta-v3-base',
] 

In [11]:
allowed_groups = {'PER', 'LOC', 'ORG', 'PERSON', 'ORGANIZATION', 'CITY', 'COUNTRY', 'REGION', 'GPE', 'PERSON'}

Извлечем из документа именнованные сущности:

In [12]:
outputs = []
for checkpoint in tqdm(checkpoints, desc="Generating models' outputs"):
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForTokenClassification.from_pretrained(checkpoint)
    
    nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy='first')

    results = []
    for chunk in chunks:
        chunk_text = article[chunk[0]: chunk[1]]
        entities = nlp(chunk_text)
        
        # removing banned entity groups
        entities = [sample for sample in entities if sample['entity_group'] in allowed_groups]
    
        # adding chunk offset to entities' positions
        for entity in entities:
            entity['start'] += chunk[0]
            entity['end'] += chunk[0]
    
        results += entities

    outputs.append(results)

Generating models' outputs:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
D:\Dev\ml_models\venv\Lib\site-packages\transformers\pipelines\token_classification.py:393: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maxim

Визуализируем выдачи моделей.

#### mbert:

In [13]:
spans = [(x['start'], x['end'], x['entity_group']) for x in outputs[0]]

show_span_box_markup(article, spans)

#### roberta:

In [14]:
spans = [(x['start'], x['end'], x['entity_group']) for x in outputs[1]]

show_span_box_markup(article, spans)

#### mdeberta:

In [15]:
spans = [(x['start'], x['end'], x['entity_group']) for x in outputs[2]]

show_span_box_markup(article, spans)

### Saving Best Model's Statistics

Можно заметить, что модель на основе mBERT немного лучше своих конкурентов (хоть и является самой маленькой по размеру):
1. меньше выделенных обрывков (пример: "Capcom U.S.A., Inc" - это 1 организация, но roberta и mdeberta выделяет только вторую половину "U.S.A., Inc");
2. roberta и mdeberta выделяют больше сущностей (например, "Капитан Коммандо" как персонажа), но границы сущностей порой определяются некорректно ("Пола Андерсона.\nВ" помечен как персонаж), mbert более консервативный в этом плане - вымышленных персонажей он так себе отлавливает, но границы сущностей определяет четко;
3. mdeberta, в частности, грешит выделением в сущность связанных слов ("филиал Capcom" - филиал считается ей как отдельная организация).

Общий итог:
- mbert имеет лучший precision;
- roberta и mdeberta лучше по recall, но precision заметно страдает.

В идеале, нужен баланс между precision/recall. Наилучший баланс наблюдается в 1ой модели. 

Исходя из этих соображений, будем сохранять статистику для модели на основе mBERT.

In [16]:
best_output = outputs[0]

for entity in best_output:
    entity['word'] = lemmatize(entity['word'])

Сохраним статистику:

In [17]:
entity_pos = pd.DataFrame(best_output)

columns = entity_pos.columns.to_list()
columns = [columns[2]] + columns[:2] + columns[3:]
entity_pos = entity_pos[columns]

entity_pos

,word,entity_group,score,start,end
0,"Capcom Co., Ltd.",ORG,0.895079,0,16
1,Осака,LOC,0.999390,180,185
2,Japan Capsule Computers,ORG,0.957055,238,261
3,Capsule,ORG,0.524911,372,379
4,Computers,ORG,0.560299,382,391
...,...,...,...,...,...
79,Capcom Co. Ltd.,ORG,0.931040,7630,7645
80,"Capcom U. S. A., Inc.",ORG,0.978601,7704,7723
81,Capcom Europe Ltd.,ORG,0.945999,7777,7795
82,Capcom,ORG,0.933932,7845,7851


In [18]:
entity_pos.to_csv('task_1/entity_positions.csv')

Посчитаем частоты сущностей:

In [19]:
frequencies = entity_pos['word'].value_counts()

entity_freq = entity_pos.copy()
entity_freq['frequency'] = entity_freq['word'].map(frequencies)

entity_freq.drop_duplicates(subset='word', keep='first', inplace=True)
entity_freq.drop(['score', 'start', 'end'], axis=1, inplace=True)
entity_freq.sort_values('frequency', ascending=False, inplace=True, ignore_index=True)

entity_freq.head(10)

,word,entity_group,frequency
0,Capcom,ORG,22
1,Япония,LOC,3
2,Clover Studio,ORG,3
3,"Capcom U. S. A., Inc.",ORG,2
4,Осака,LOC,2
5,"Flagship Co., Ltd",ORG,1
6,Лондон,LOC,1
7,CE Europe Ltd.,ORG,1
8,"Capcom Charbo Co., Ltd.",ORG,1
9,"Capcom Co., Ltd.",ORG,1


In [20]:
entity_freq.to_csv('task_1/entity_frequencies.csv')

### Conclusion

В целом, качество извлечения именованных сущностей у mbert лучшее среди 3 моделей. Однако не все так гладко:
1. модель хорошо теггирует реальных людей (Джеки Чан), но с вымышленными персонажами может ошибаться (например, Капитан Коммандо, хотя Рю был выделен корректно - возможно, дело в популярности этих персонажей, про Рю из Street Fighter скорее всего что-то да было в их датасете);
2. малоизвестные студии (M-Two, K2) плохо теггируются (либо обрывочно, либо совсем никак);
3. неоднозначные кейсы (MegaMan, это и название игры, и персонаж) не ловятся.

Возможно, разница между моделями лежит в датасетах, на которых они обучались. Крупные roberta и mdeberta ловят больше сущностей, но и ошибаются чаще, даже в якобы простых случаях.